In [1]:
import numpy as np
import pprint
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [2]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

In [3]:
def value_iteration(env, theta=0.0001, discount_factor=1.0):
    """
    Value Iteration Algorithm.
    
    Args:
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
        
    Returns:
        A tuple (policy, V) of the optimal policy and the optimal value function.        
    """
    

    def lookahead(s):
        # one step lookahead from state s
        qs = np.zeros(env.nA)
        for a in range(env.nA):
            # determine q(s,a)
            for prob, next_state, reward, done in env.P[s][a]:
                qs[a] += prob*(reward + discount_factor*prev_V[next_state])
        return qs
    
    V = np.zeros(env.nS)
    policy = np.zeros([env.nS, env.nA])
    
    # find V*
    while True:
        # update V for every state
        prev_V = V
        V = np.zeros(env.nS)
        
        for s in range(env.nS):
            # check every action to determine the optimal option
            qs = lookahead(s)
            # update the value of the state
            V[s] = np.max(qs)
            
        if np.max(np.abs(prev_V-V)) < theta:
            break
        
        
    # find the policy
    for s in range(env.nS):
        # check every action to determine the optimal option
        
        
        qs = lookahead(s)
        
        # determine the policy
        policy[s] = np.zeros(env.nA)
        policy[s][np.argmax(qs)] = 1
        
        # breaking ties is pointless. we should just find the best deterministic option.
        # previous version that did break ties:

#         best_as = np.nonzero(qs==np.max(qs))[0] # check for ties
        
#         # determine the policy
#         policy[s] = np.zeros(env.nA)
#         for a in best_as:
#             policy[s][a] = 1/len(best_as)
        

    return policy, V

In [4]:
policy, v = value_iteration(env)

print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")

Policy Probability Distribution:
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]

Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]

Value Function:
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]

Reshaped Grid Value Function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]



In [5]:
# Test the value function
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)